In [1]:
%pylab nbagg
from tvb.simulator.lab import *
import pandas as pd
import h5py

from tvb.simulator.lab import *
import h5py
import os
import matplotlib.pyplot as plt

import scipy 
import scipy.fftpack
import seaborn as sns

import scipy 
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import signal
from scipy.integrate import simps
from fooof import FOOOF

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#depressed brain 
Q_health = np.repeat(1, 76)
oscilator = models.WilsonCowan(Q = Q_health)

In [4]:
white_matter = connectivity.Connectivity.from_file()
white_matter.speed = numpy.array([3.0])

WARNING  File 'hemispheres' not found in ZIP.


In [5]:
white_matter_coupling = coupling.Linear(a=numpy.array([0.0039]))

In [6]:
#Initialise some Monitors with period in physical time
mon_raw = monitors.Raw(period = 0.9765625)
mon_tavg = monitors.TemporalAverage(period = 0.9765625)

#Bundle them
what_to_watch = (mon_raw, mon_tavg)

In [7]:
#important variables of dataset
sample_period = 0.0009765625 #seconds         
sf = 1/sample_period #sampling frequency 
sample_rate = 1024

#sns.set(font_scale=1.2)
dt = 0.01

In [8]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Wilson Cowan/DATA")
#import connectivity file to know which region corresponds to which index in the matrices
filename = "Connectivity.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    regions_list = list(f['region_labels'])

regions = np.array(regions_list)


Keys: <KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>


In [9]:
def powercalc(data, low, high, sf, win):
    freqs, psd = signal.welch(data, sf, nperseg=win)
    freq_res = freqs[1] - freqs[0]
    total_power = simps(psd, dx=freq_res)
    idx= np.logical_and(freqs >= low, freqs <= high)
    power = simps(psd[idx], dx=freq_res)
    return power

In [30]:
#loop  start om 9u56
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_freqs/Peak"



peak_right = {}
peak_left = {}

win = 4 * sf
sf = 1024

freq_range = [8,12] 

indices = [7,  13, 15, 16, 17,18, 19, 20, 21, 30, 31, 33,35, 36, 45, 51, 53, 54, 55, 56, 57, 58, 59, 68, 69, 71 , 73,74]
NS_arr = np.arange(0,26, 0.25)

counter = 0
loop = 1

nsubs = 5 #101

for i in range(nsubs): 
    if i == 0:
        pass 
    else:
        value = 10**-6 #noise dispertion
        NS = NS_arr[i] #changing noise seed
        print(loop)
        
        #create simulator 
        sim = simulator.Simulator(model = oscilator, connectivity = white_matter,
                          coupling = white_matter_coupling, 
                          integrator = integrators.EulerStochastic(dt=0.01220703125, noise=noise.Additive(noise_seed = int(NS), nsig=numpy.array([value]))),
                        monitors =  what_to_watch)

        sim.configure()

        print("check1")
        
        #Perform the simulation
        tavg_data = []
        tavg_time = []

        for raw, tavg in sim(simulation_length=2500):
    
            if not tavg is None:
                tavg_time.append(tavg[0])
                tavg_data.append(tavg[1])
    
        #Make the lists numpy.arrays for easier use.
        RAW = numpy.array(tavg_data)
        RAW_real = RAW[:, 0, :, 0]
        
        if loop == 1:
            sf = len(RAW_real)/2.5 #calculate sampling frequency
            print(sf)
            print(len(tavg_time))
            print(shape(RAW_real))
        
        print("check2")
        #delete the first 500ms
        list_deleterows = [*range(0,500,1)]
        RAW_real = np.delete(RAW_real, list_deleterows, 0)
        tavg_time =  np.delete(tavg_time, list_deleterows, 0)
        
        #save power spectrum of left DLPFC
        leftDLPFC = RAW_real[:,56]
        
        freqs_left, psd_left = signal.welch(leftDLPFC, sf, nperseg = win)
        
        fm = FOOOF(peak_width_limits=[1, 8])
        fm.fit(freqs_left, psd_left, freq_range)
        peaks_left = fm.get_params('peak_params')
        peak_left["data" + str(i)] = peaks_left[0][0]
        
        print(peaks_left)
        
        #save power spectrum of rigth DLPFC
        rightDLPFC = RAW_real[:,18]
        
        freqs_right, psd_right = signal.welch(rightDLPFC, sf, nperseg = win)
        
        fm = FOOOF(peak_width_limits=[1, 8])
        fm.fit(freqs_right, psd_right, freq_range)
        peaks_right = fm.get_params('peak_params')
        peak_right["data" + str(i)] = peaks_right[0][0]


                
        loop = loop +1
    


#make dataset containing all 100 left DLPFC time series 
namelist6 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name6 = peak_left["data"+str(i)] 
        namelist6.append(name6)

nametup6 = tuple(namelist6)
leftpeaks = np.column_stack(nametup6)

#make dataset containing all 100 left DLPFC time series 
namelist6 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name6 = peak_right["data"+str(i)] 
        namelist6.append(name6)

nametup6 = tuple(namelist6)
rightpeaks = np.column_stack(nametup6)



file_name = mydir + "/health_alpha_left_peaks.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=leftpeaks)
h5f.close()

file_name = mydir + "/health_alpha_right_peaks.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=rightpeaks)
h5f.close()



1
check1
1024.0
2560
(2560, 76)
check2
[[10.55769332  0.97024553  1.        ]]
2
check1
check2
[[10.55769332  0.97024553  1.        ]]
3
check1
check2
[[10.55769332  0.97024553  1.        ]]
4
check1
check2
[[9.761118   0.54484803 1.        ]]
